# Library Book Management System

In [1]:
from tkinter import *
import sqlite3
import tkinter.ttk as ttk
import tkinter.messagebox as tkMessageBox

root = Tk()
root.title('Library Book Management System')

# Methods
def Database():
    global conn, cursor
    conn = sqlite3.connect('Book_Management.db')
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS member(booktitle TEXT, author TEXT, year INTEGER, isbn INTEGER)')

def Display():
    tree.delete(*tree.get_children())
    Database()
    cursor.execute("SELECT * FROM member ORDER BY booktitle ASC")
    fetch = cursor.fetchall()
    for data in fetch:
        if len(data) >= 4:
            tree.insert('', 'end', values=(data[0], data[1], data[2], data[3]))
        else:
            print("Invalid data format:", data)
    cursor.close()
    txt_result.config(text="All Data Displayed !", fg="green")
        # Call the Add function to update the treeview
        

def Search():
    tree.delete(*tree.get_children())
    Database()

    # Get the booktitle to search
    search_booktitle = str(BOOKTITLE.get())

    # Check if the search_booktitle is empty
    if not search_booktitle:
        txt_result.config(text="Please enter a booktitle for search", fg="red")
        return

    # Execute the SELECT query with a WHERE clause
    cursor.execute("SELECT * FROM member WHERE booktitle LIKE ?", ('%' + search_booktitle + '%',))
    fetch = cursor.fetchall()

    if not fetch:
        txt_result.config(text="Book not found", fg="red")
    else:
        for data in fetch:
            if len(data) >= 4:
                tree.insert('', 'end', values=(data[0], data[1], data[2], data[3]))
            else:
                print("Invalid data format:", data)
        txt_result.config(text="Successfully fetched data !", fg="green")

    cursor.close()


def Add():
    if BOOKTITLE.get() == "" or AUTHOR.get() == "" or YEAR.get() == "" or ISBN.get() == "":
        txt_result.config(text="Please enter all the fields", fg="red")
    else:
        Database()
        cursor.execute("INSERT INTO member(booktitle, author, year, isbn) VALUES (?,?,?,?)",
                       (str(BOOKTITLE.get()), str(AUTHOR.get()), str(YEAR.get()), str(ISBN.get())))
        conn.commit()
        BOOKTITLE.set("")
        AUTHOR.set("")
        YEAR.set("")
        ISBN.set("")
        cursor.close()
        conn.close()
        txt_result.config(text="Record Inserted !", fg="green")

def Issue():
    selected_item = tree.selection()
    
    if not selected_item:
        txt_result.config(text="Please select a book to Issue", fg="red")
        return
    
    Database()
    
    for item in selected_item:
        booktitle_to_issue = tree.item(item, 'values')[0]
        cursor.execute("SELECT * FROM member WHERE booktitle=?",(booktitle_to_issue,))
        
    conn.commit()
    conn.close()
    
    
    result = tkMessageBox.askquestion('Confirmation', 'Do you want to issue this book?', icon='question')
    if result == 'yes':
        txt_result.config(text="Book Issued", fg="green")


def Delete():
    selected_item = tree.selection()

    if not selected_item:
        txt_result.config(text="Please select a book to delete", fg="red")
        return

    Database()

    for item in selected_item:
        booktitle_to_delete = tree.item(item, 'values')[0]
        cursor.execute("DELETE FROM member WHERE booktitle=?", (booktitle_to_delete,))

    conn.commit()
    conn.close()

    result = tkMessageBox.askquestion('Confirmation','Are you sure to delete ? (y/n)', icon='question')
    if result == 'yes':
        tree.delete(*selected_item)
        Display()
        txt_result.config(text="Successfully deleted data !", fg="green")
    else:
        txt_result.config(text="Book not deleted", fg="green")    


def Exit():
    result = tkMessageBox.askquestion('Confirmation','Do you want to exit ? (y/n)', icon='warning')
    if result == 'yes':
        root.destroy()
        exit()

# Variable declaration
BOOKTITLE = StringVar()
AUTHOR = StringVar()
YEAR = StringVar()
ISBN = StringVar()

# Frame
Top = Frame(root, width=900, height=50, bd=8, relief='raise')
Top.pack(side=TOP)
Left = Frame(root, width=300, height=500, bd=8, relief='raise')
Left.pack(side=LEFT)
Right = Frame(root, width=300, height=500, bd=8, relief='raise')
Right.pack(side=RIGHT)

Forms = Frame(Left, width=300, height=450)
Forms.pack(side=TOP)

Buttons = Frame(Left, width=300, height=100, bd=8, relief='raise')
Buttons.pack(side=BOTTOM)

# Labels Widget
txt_title = Label(Top, width=900, font=('arial', 24), text='Library Book Management System')
txt_title.pack()
txt_booktitle = Label(Forms, text="Booktitle:", font=('arial', 16), bd=15)
txt_booktitle.grid(row=0, stick='e')
txt_author = Label(Forms, text='Author:', font=('arial', 16), bd=15)
txt_author.grid(row=1, stick='e')
txt_year = Label(Forms, text='Year:', font=('arial', 16), bd=15)
txt_year.grid(row=2, stick='e')
txt_isbn = Label(Forms, text='ISBN:', font=('arial', 16), bd=15)
txt_isbn.grid(row=3, stick='e')
txt_result = Label(Buttons)
txt_result.pack(side=TOP)

# Entry Widget
booktitle = Entry(Forms, textvariable=BOOKTITLE, width=30)
booktitle.grid(row=0, column=1)
author = Entry(Forms, textvariable=AUTHOR, width=30)
author.grid(row=1, column=1)
year = Entry(Forms, textvariable=YEAR, width=30)
year.grid(row=2, column=1)
isbn = Entry(Forms, textvariable=ISBN, width=30)
isbn.grid(row=3, column=1)

# BUTTONS WIDGET
btn_create = Button(Buttons, width=10, text='Display', command=Display)
btn_create.pack(side=LEFT)
btn_create = Button(Buttons, width=10, text='Search', command=Search)
btn_create.pack(side=LEFT)
btn_create = Button(Buttons, width=10, text='Add', command=Add)
btn_create.pack(side=LEFT)
btn_create = Button(Buttons, width=10, text='Issue', command=Issue)
btn_create.pack(side=LEFT)
btn_create = Button(Buttons, width=10, text='Delete', command=Delete)
btn_create.pack(side=LEFT)
btn_create = Button(Buttons, width=10, text='Exit', command=Exit)
btn_create.pack(side=LEFT)

# LIST WIDGET
scrollbary = Scrollbar(Right, orient=VERTICAL)
scrollbarx = Scrollbar(Right, orient=HORIZONTAL)

tree = ttk.Treeview(Right, columns=("Booktitle", "Author", "Year", "ISBN"))

scrollbary.config(command=tree.yview)
scrollbary.pack(side=RIGHT, fill=Y)
scrollbarx.config(command=tree.xview)
scrollbarx.pack(side=BOTTOM, fill=X)
tree.heading('Booktitle', text="Booktitle", anchor=W)
tree.heading('Author', text="Author", anchor=W)
tree.heading('Year', text="Year", anchor=W)
tree.heading('ISBN', text="ISBN", anchor=W)
tree.column('#0', stretch=NO, minwidth=0, width=0)
tree.column('#1', stretch=NO, minwidth=0, width=200)
tree.column('#2', stretch=NO, minwidth=0, width=200)
tree.column('#3', stretch=NO, minwidth=0, width=200)
tree.pack()

# INITIALIZATION
if __name__ == '__main__':
    root.mainloop()
